# PS 4 Iddrisu Kambala Mohammed

In [47]:
##importing necessary packages 
import pandas as pd
import numpy as np
import scipy.optimize as opt
import time
%matplotlib inline
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy.stats as stats
import math

In [48]:
#reading in the Stata File
hpv_data = pd.read_stata('PS4_data.dta')
hpv_data.head(n=6)

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
0,1,1967,1,Head,1200.0,2000.0,NaN,NaN,0,NaN,...,5614.0,1.0,2.0,52.0,46.0,1.0,2.0,8.0,8.0,0.0
1,2,1967,2,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,56.0,57.0,1.0,2.0,3.0,3.0,0.0
2,3,1967,3,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,77.0,64.0,1.0,2.0,NaN,3.0,0.0
3,4,1967,4,Head,1560.0,0.0,NaN,NaN,6,1.0,...,3280.0,1.0,2.0,45.0,44.0,1.0,2.0,8.0,5.0,0.0
4,5,1967,5,Head,2500.0,2000.0,NaN,NaN,3,1.0,...,7900.0,1.0,2.0,24.0,22.0,1.0,2.0,10.0,9.0,0.0
5,6,1967,6,Head,2230.0,120.0,NaN,NaN,2,1.0,...,6269.0,1.0,2.0,49.0,46.0,1.0,2.0,8.0,13.0,0.0


In [49]:
hpv_data.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
count,123786.000000,123786.000000,123786.000000,123786.000000,123786.000000,9.023300e+04,48496.000000,123786.000000,90603.000000,123656.000000,...,1.237860e+05,123786.000000,80758.0,123786.000000,80758.000000,123786.000000,80758.000000,122809.000000,80091.000000,123786.000000
mean,1494.639475,1984.831273,3271.379429,1679.269897,633.026917,4.211505e+04,22026.289062,0.843771,1.098220,1.129731,...,3.012258e+04,1.233072,2.0,45.545547,41.390785,39.620201,55.346169,12.666091,12.720081,0.557690
std,838.901790,9.836212,2277.056058,1061.704712,878.422791,4.670424e+04,21336.107422,1.182829,0.356161,0.394627,...,4.588795e+04,0.422940,0.0,17.623671,14.786721,69.003265,77.864296,2.917721,2.422607,0.265198
min,1.000000,1967.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,-1.324040e+05,1.000000,2.0,16.000000,13.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,772.000000,1977.000000,1444.000000,832.000000,0.000000,1.979858e+04,8016.247070,0.000000,1.000000,1.000000,...,7.700000e+03,1.000000,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.362158
50%,1517.000000,1985.000000,2984.000000,1976.000000,0.000000,3.460022e+04,18122.412109,0.000000,1.000000,1.000000,...,1.900000e+04,1.000000,2.0,42.000000,39.000000,3.000000,3.000000,12.000000,12.000000,0.599887
75%,2224.000000,1993.000000,4763.000000,2350.000000,1454.000000,5.267309e+04,30256.060547,2.000000,1.000000,1.000000,...,3.910775e+04,1.000000,2.0,58.000000,51.000000,22.000000,170.000000,15.000000,14.000000,0.786908
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,856942.062500,11.000000,8.000000,8.000000,...,3.660000e+06,2.000000,2.0,102.000000,95.000000,227.000000,231.000000,17.000000,17.000000,0.928007


In [50]:
hpv_data['wage'] = hpv_data['hlabinc'] / hpv_data['hannhrs'] ##calculating hourly wage
hpv_data['logwage'] = np.log(hpv_data['wage']) ##log of hourly wage
hpv_data['agesq'] = (hpv_data['age']*hpv_data['age']) ##age squared variable
hpv_data

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce,logwage,agesq
0,1,1967,1,Head,1200.0,2000.0,NaN,NaN,0,NaN,...,2.0,52.0,NaN,1.0,2.0,8.0,8.0,0.000000,NaN,2704.0
1,2,1967,2,Head,0.0,0.0,NaN,NaN,0,NaN,...,2.0,56.0,NaN,1.0,2.0,3.0,3.0,0.000000,NaN,3136.0
2,3,1967,3,Head,0.0,0.0,NaN,NaN,0,NaN,...,2.0,77.0,NaN,1.0,2.0,NaN,3.0,0.000000,NaN,5929.0
3,4,1967,4,Head,1560.0,0.0,NaN,NaN,6,1.0,...,2.0,45.0,NaN,1.0,2.0,8.0,5.0,0.000000,NaN,2025.0
4,5,1967,5,Head,2500.0,2000.0,NaN,NaN,3,1.0,...,2.0,24.0,NaN,1.0,2.0,10.0,9.0,0.000000,NaN,576.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123781,2067,2002,7862,Head,2088.0,0.0,25861.861328,NaN,0,1.0,...,NaN,22.0,12.385949,35.0,NaN,12.0,NaN,0.928007,2.516563,484.0
123782,1430,2002,7866,Head,0.0,2080.0,NaN,37715.214844,0,1.0,...,2.0,42.0,NaN,5.0,171.0,15.0,14.0,0.928007,NaN,1764.0
123783,2354,2002,7867,Head,521.0,0.0,33404.902344,NaN,1,1.0,...,NaN,24.0,64.116898,31.0,NaN,12.0,NaN,0.928007,4.160708,576.0
123784,1634,2002,7868,Head,2288.0,0.0,42025.523438,NaN,0,1.0,...,NaN,47.0,18.367798,4.0,NaN,12.0,NaN,0.928007,2.910599,2209.0


In [51]:
hpv_data.hrace.describe() ## inspecting the race variable: there is no hispanic

count    123656.000000
mean          1.129731
std           0.394627
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           8.000000
Name: hrace, dtype: float64

In [52]:
## dropping missing values for relevant variables and generating race dummies (there is no Hispanic)
hpv_data = hpv_data[hpv_data.hsex.isnull() != True ]
hpv_data = hpv_data[hpv_data.hrace.isnull() != True ]
hpv_data = hpv_data[hpv_data.hyrsed.isnull() != True ]
hpv_data = hpv_data[hpv_data.hlabinc.isnull() != True ]
hpv_data = hpv_data[hpv_data.hannhrs.isnull() != True ]
hpv_data = hpv_data[hpv_data.logwage.isnull() != True ]
hpv_data = hpv_data[hpv_data.wage.isnull() != True ]

hpv_data['Black'] = hpv_data['hrace'] == 2
hpv_data['OtherRace'] = hpv_data['hrace'] == 3 | 8
hpv_data

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,wage,hpersno,wpersno,hyrsed,wyrsed,pce,logwage,agesq,Black,OtherRace
11161,402,1971,1,Head,1523.0,0.0,62928.707031,NaN,0,1.0,...,41.318916,1.0,2.0,12.0,12.0,0.247121,3.721320,2601.0,False,False
11162,446,1971,2,Head,520.0,0.0,1618.640747,NaN,0,NaN,...,3.112771,1.0,NaN,17.0,NaN,0.247121,1.135513,3844.0,False,False
11164,461,1971,4,Head,2010.0,0.0,22660.970703,NaN,0,1.0,...,11.274115,1.0,2.0,5.0,5.0,0.247121,2.422509,3025.0,False,False
11165,462,1971,5,Head,1960.0,0.0,12949.125977,NaN,0,NaN,...,6.606697,1.0,2.0,5.0,8.0,0.247121,1.888084,3481.0,False,False
11166,1126,1971,8,Head,2860.0,0.0,29337.865234,NaN,1,NaN,...,10.257995,1.0,2.0,16.0,12.0,0.247121,2.328057,625.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123780,826,2002,7857,Head,2064.0,0.0,86206.203125,NaN,1,1.0,...,41.766571,178.0,NaN,13.0,NaN,0.928007,3.732096,625.0,False,False
123781,2067,2002,7862,Head,2088.0,0.0,25861.861328,NaN,0,1.0,...,12.385949,35.0,NaN,12.0,NaN,0.928007,2.516563,484.0,False,False
123783,2354,2002,7867,Head,521.0,0.0,33404.902344,NaN,1,1.0,...,64.116898,31.0,NaN,12.0,NaN,0.928007,4.160708,576.0,False,False
123784,1634,2002,7868,Head,2288.0,0.0,42025.523438,NaN,0,1.0,...,18.367798,4.0,NaN,12.0,NaN,0.928007,2.910599,2209.0,False,False


In [53]:
##cleaning & creating relevant variables

hpv_data = hpv_data[(hpv_data['hsex'] == 1)] ##restrciting to male householders
min_age = 21 ##lower age limit
max_age = 65 ##upper age limit
min_wage = 7 ##cut off wage/hr
years = [1971, 1980, 1990, 2000] ##relevant years
hpv_data = hpv_data[ ['age', 'agesq', 'hlabinc', 'hannhrs', 
                      'hyrsed', 'logwage', 'wage', 'year', 'OtherRace', 'Black' ] ] ##keeping just relevant variables
hpv_data = hpv_data[ (hpv_data.wage > min_wage) & (hpv_data.age >= min_age) & \
                    (hpv_data.age <= max_age) & (hpv_data.logwage != np.inf)]
hpv_data['const'] = 1 ##a constant for the estimation
hpv_data

,age,agesq,hlabinc,hannhrs,hyrsed,logwage,wage,year,OtherRace,Black,const
11161,51.0,2601.0,62928.707031,1523.0,12.0,3.721320,41.318916,1971,False,False,1
11164,55.0,3025.0,22660.970703,2010.0,5.0,2.422509,11.274115,1971,False,False,1
11166,25.0,625.0,29337.865234,2860.0,16.0,2.328057,10.257995,1971,False,False,1
11170,63.0,3969.0,19678.625000,1960.0,6.0,2.306588,10.040114,1971,False,False,1
11173,39.0,1521.0,76885.437500,2400.0,16.0,3.466848,32.035599,1971,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...
123775,36.0,1296.0,4310.310059,16.0,16.0,5.596177,269.394379,2002,False,False,1
123777,51.0,2601.0,34482.480469,2080.0,12.0,2.808084,16.578115,2002,False,False,1
123783,24.0,576.0,33404.902344,521.0,12.0,4.160708,64.116898,2002,False,False,1
123784,47.0,2209.0,42025.523438,2288.0,12.0,2.910599,18.367798,2002,False,False,1


In [54]:
hpv_data.describe() ##everything looks okay!!

,age,agesq,hlabinc,hannhrs,hyrsed,logwage,wage,year,const
count,64107.000000,64107.000000,6.410700e+04,64107.000000,64107.000000,64107.000000,64107.000000,64107.000000,64107.0
mean,39.020325,1644.557617,5.088534e+04,2196.280762,13.432885,2.983948,23.808784,1986.251564,1.0
std,11.068608,925.334167,5.118344e+04,638.316528,2.456135,0.550327,25.919069,8.768032,0.0
min,21.000000,441.000000,1.666980e+01,2.000000,1.000000,1.945946,7.000252,1971.000000,1.0
25%,30.000000,900.000000,2.862519e+04,1933.000000,12.000000,2.596890,13.421934,1979.000000,1.0
50%,37.000000,1369.000000,4.193501e+04,2141.000000,13.000000,2.959912,19.296280,1986.000000,1.0
75%,47.000000,2209.000000,5.964020e+04,2500.000000,16.000000,3.304278,27.228870,1993.000000,1.0
max,65.000000,4225.000000,3.771521e+06,5840.000000,17.000000,7.531037,1865.037964,2002.000000,1.0


# Maximum Likelihood Estimation

In [55]:
def mle(b, hpv_data, t):
    '''
    this allows to compute the LLF
    
    Args:
        B: coeffficients of interest
        sigma (float): the standard deviation generated from the coefficients
        hpv_data: the HPV dataset
        t: = 1971, 1980, 1990, 2000
        
    Returns:
        LL_negative (float): -LL
    '''
    mle_hpv_data = hpv_data[hpv_data['year'].isin(t)]
    ##let's define the positions of the parameters
    cons = b[0]
    beta1 = b[1]
    beta2 = b[2]
    beta3 = b[3]
    beta4 = b[4]
    beta5 = b[5]
    sigma = b[6]
    
    N = len(mle_hpv_data['logwage'])
    ##we now define the linear model as given in the question (Note: there is no hispanic in the sample)
    logwage = cons + beta1 * mle_hpv_data['hyrsed'] + beta2 * mle_hpv_data['age']+ beta3 * mle_hpv_data['agesq'] + beta4 * mle_hpv_data['Black'] + beta5 * mle_hpv_data['OtherRace']
    
    ##we define the error term
    e = mle_hpv_data['logwage'] - logwage
    
    ##and finally the negative LL
    LL_negative = (N/2) * np.log(sigma ** 2) + (N/2) * np.log(2 * math.pi) + (np.sum(e ** 2)) / (2 * sigma ** 2)
    
    return(LL_negative)

In [56]:
##putting our initial guesses and estimating
init_guess = [1, 0.05, 0.04, -0.01, -0.02, -0.01, 0.5] # initial coefficient guesses
results = []
for t in years:
    params = (hpv_data, [t])
    result = opt.minimize(mle, init_guess, args = params , method = 'BFGS', tol = 1e-10, options = {'maxiter': 5000})
    results.append(result.x)
    print(t, result.x)

1971 [ 1.00000000e+00  5.99374177e-02  5.12457036e-02 -4.91232087e-04
 -1.71200145e-01 -1.00000000e-02  4.13954504e-01]
1980 [ 1.00000000e+00  7.03553199e-02  4.27215008e-02 -3.65597214e-04
 -1.18964316e-01 -1.00000000e-02  4.47127931e-01]
1990 [ 1.00000000e+00  7.94867753e-02  3.12451288e-02 -2.07383066e-04
 -1.88373563e-01 -1.00000000e-02  4.83963103e-01]
2000 [ 1.00000000e+00  1.28018887e-01  6.00799251e-03  5.12171078e-05
 -2.29170927e-01 -1.00000000e-02  5.38090301e-01]


Interpretation: Beta1 can be interpreted as the percentage change in hourly wage as a result of a one-year increase in years of education. If we compare the coefficients for the four years, we can observe that the economic return to school observed a steady increase since 1971. While one more year of education was associated with about 6 percent increase in hourly wage in 1971, it rose to about 12.8 percent in 2000. 

# Running OLS

In [33]:
##1971: OLS .06, MLE .0599
hpv_1971 = hpv_data[hpv_data['year'] == 1971]
x_1971 = hpv_1971[['hyrsed', 'age', 'agesq', 'Black', 'OtherRace']]
y_1971 = hpv_1971['logwage']
x_1971 = sm.add_constant(x_1971)
result_1971 = sm.OLS(y_1971, x_1971.astype(float)).fit()

print(result_1971.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     146.3
Date:                Tue, 05 Oct 2021   Prob (F-statistic):          2.26e-107
Time:                        01:14:32   Log-Likelihood:                -892.48
No. Observations:                1679   AIC:                             1795.
Df Residuals:                    1674   BIC:                             1822.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5673      0.125      4.550      0.0

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [34]:
##1980:OLS .064, MLE .70
hpv_1980 = hpv_data[hpv_data['year'] == 1980]
x_1980 = hpv_1980[['hyrsed', 'age', 'agesq', 'Black', 'OtherRace']]
y_1980 = hpv_1980['logwage']
x_1980 = sm.add_constant(x_1980)
result_1980 = sm.OLS(y_1980, x_1980.astype(float)).fit()

print(result_1980.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     134.3
Date:                Tue, 05 Oct 2021   Prob (F-statistic):          2.31e-102
Time:                        01:17:47   Log-Likelihood:                -1302.5
No. Observations:                2124   AIC:                             2615.
Df Residuals:                    2119   BIC:                             2643.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9205      0.133      6.936      0.0

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [36]:
##1990: OLS .09, MLE .08
hpv_1990 = hpv_data[hpv_data['year'] == 1990]
x_1990 = hpv_1990[['hyrsed', 'age', 'agesq', 'Black', 'OtherRace']]
y_1990 = hpv_1990['logwage']
x_1990 = sm.add_constant(x_1990)
result_1990 = sm.OLS(y_1990, x_1990.astype(float)).fit()

print(result_1990.summary())

                            OLS Regression Results                            
Dep. Variable:                logwage   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     184.1
Date:                Tue, 05 Oct 2021   Prob (F-statistic):          1.98e-136
Time:                        01:22:17   Log-Likelihood:                -1539.8
No. Observations:                2231   AIC:                             3090.
Df Residuals:                    2226   BIC:                             3118.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3855      0.149      2.583      0.0

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [43]:
##2000: OLS .11, MLE .13
hpv_2000 = hpv_data[hpv_data['year'] == 2000]
x_2000 = hpv_2000[['hyrsed', 'age', 'agesq', 'Black', 'OtherRace']]
y_2000 = hpv_2000['logwage']
x_1990 = sm.add_constant(x_2000)
result_2000 = sm.OLS(y_2000, x_2000.astype(float)).fit()

print(result_2000.summary())

                                 OLS Regression Results                                
Dep. Variable:                logwage   R-squared (uncentered):                   0.971
Model:                            OLS   Adj. R-squared (uncentered):              0.971
Method:                 Least Squares   F-statistic:                          2.375e+04
Date:                Tue, 05 Oct 2021   Prob (F-statistic):                        0.00
Time:                        01:37:01   Log-Likelihood:                         -2230.1
No. Observations:                2813   AIC:                                      4468.
Df Residuals:                    2809   BIC:                                      4492.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
